# Schematic

A schematic is a graph representation of your circuit.

For complex circuits, a schematic allows you to create symbols and hierarchy levels to represent your circuit.

Having a schematic allows you to also ensure that your layout matches you schematic (design intent).

There are many schematic capture tools out there:

- Qucs-s: for RF.
- Xschem: for analog circuits.
- Lumerical interconnect: for photonic circuits.

These tools allow you to both create schematics with your mouse or by code.

gdsfactory also allows you to create complex Schematics directly from python with a very simple interface.

In [1]:
import gdsfactory as gf
import gdsfactory.schematic as gt

Lets create a MZI lattice of 3 elements.

In [6]:
s = gt.Schematic()
s.add_instance("mzi1", gt.Instance(component=gf.c.mzi(delta_length=10)))
s.add_instance("mzi2", gt.Instance(component=gf.c.mzi(delta_length=100)))
s.add_instance("mzi3", gt.Instance(component=gf.c.mzi(delta_length=200)))
s.add_placement("mzi1", gt.Placement(x=000, y=0))
s.add_placement("mzi2", gt.Placement(x=100, y=100))
s.add_placement("mzi3", gt.Placement(x=200, y=0))
s.add_net(gt.Net(p1="mzi1,o2", p2="mzi2,o2"))
s.add_net(gt.Net(p1="mzi2,o2", p2="mzi3,o1"))
g = s.netlist
g

Netlist(pdk='', instances={'mzi1': Instance(component='mzi', settings={'delta_length': 10, 'length_y': 2.0, 'length_x': 0.1, 'bend': 'bend_euler', 'straight': 'straight', 'splitter': 'mmi1x2', 'with_splitter': True, 'port_e1_splitter': 'o2', 'port_e0_splitter': 'o3', 'port_e1_combiner': 'o2', 'port_e0_combiner': 'o3', 'port1': 'o1', 'port2': 'o2', 'nbends': 2, 'cross_section': 'strip', 'mirror_bot': False, 'add_optical_ports_arms': False, 'min_length': 0.01, 'auto_rename_ports': True}, info={}, array=None), 'mzi2': Instance(component='mzi', settings={'delta_length': 100, 'length_y': 2.0, 'length_x': 0.1, 'bend': 'bend_euler', 'straight': 'straight', 'splitter': 'mmi1x2', 'with_splitter': True, 'port_e1_splitter': 'o2', 'port_e0_splitter': 'o3', 'port_e1_combiner': 'o2', 'port_e0_combiner': 'o3', 'port1': 'o1', 'port2': 'o2', 'nbends': 2, 'cross_section': 'strip', 'mirror_bot': False, 'add_optical_ports_arms': False, 'min_length': 0.01, 'auto_rename_ports': True}, info={}, array=None), 

You can also create a splitter tree.

In [8]:
s = gt.Schematic()
s.add_instance("s11", gt.Instance(component=gf.c.mmi1x2()))
s.add_instance("s21", gt.Instance(component=gf.c.mmi1x2()))
s.add_instance("s22", gt.Instance(component=gf.c.mmi1x2()))
s.add_placement("s11", gt.Placement(x=000, y=0))
s.add_placement("s21", gt.Placement(x=100, y=+50))
s.add_placement("s22", gt.Placement(x=100, y=-50))
s.add_net(gt.Net(p1="s11,o2", p2="s21,o1"))
s.add_net(gt.Net(p1="s11,o3", p2="s22,o1"))
g = s.netlist
g

Netlist(pdk='', instances={'s11': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None), 's21': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None), 's22': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None)}, placements={'s11': Placement(x=0.0, y=0.0, xmin=None, ymin=None, xmax=None, ymax=None, dx=0, dy=0, port=None, rotation=0, mirror=False), 's21': Placement(x=100.0, y=50.0, xmin=None, ymin=None, xmax=None, ymax=None, dx=0, dy=0, port=None, rotation=0, mirror=False), 's22': P

The nice thing is that you can abstract it to have as many levels as you need.

In [10]:
splitter = gf.components.mmi1x2()
n = 3
dx = 100
dy = 100
s = gt.Schematic()

for col in range(n):
    rows = 2**col
    for row in range(rows):
        s.add_instance(f"s{col}{row}", gt.Instance(component=splitter))
        s.add_placement(
            f"s{col}{row}", gt.Placement(x=col * dx, y=(row - rows / 2) * dy)
        )
        if col < n - 1:
            s.add_net(gt.Net(p1=f"s{col}{row},o2", p2=f"s{col+1}{2*row},o1"))
            s.add_net(gt.Net(p1=f"s{col}{row},o3", p2=f"s{col+1}{2*row+1},o1"))


g = s.netlist
g

Netlist(pdk='', instances={'s00': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None), 's10': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None), 's11': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None), 's20': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None), 's21': Instance(component='mmi1x2', setti

In [11]:
splitter = gf.components.mmi1x2()
n = 5
dx = 100
dy = 100
s = gt.Schematic()

for col in range(n):
    rows = 2**col
    for row in range(rows):
        s.add_instance(f"s{col}{row}", gt.Instance(component=splitter))
        s.add_placement(
            f"s{col}{row}", gt.Placement(x=col * dx, y=(row - rows / 2) * dy)
        )
        if col < n - 1:
            s.add_net(gt.Net(p1=f"s{col}{row},o2", p2=f"s{col+1}{2*row},o1"))
            s.add_net(gt.Net(p1=f"s{col}{row},o3", p2=f"s{col+1}{2*row+1},o1"))


g = s.netlist
g

Netlist(pdk='', instances={'s00': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None), 's10': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None), 's11': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None), 's20': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None), 's21': Instance(component='mmi1x2', setti

In [12]:
dict(s.netlist)

{'pdk': '',
 'instances': {'s00': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None),
  's10': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None),
  's11': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None),
  's20': Instance(component='mmi1x2', settings={'width_taper': 1.0, 'length_taper': 10.0, 'length_mmi': 5.5, 'width_mmi': 2.5, 'gap_mmi': 0.25, 'taper': 'taper', 'straight': 'straight', 'cross_section': 'strip'}, info={}, array=None),
  's21': Instance(component='mmi1x2

In [13]:
import yaml

In [14]:
yaml_component = yaml.dump(s.netlist.model_dump(exclude_none=True))
print(yaml_component)

connections: {}
info: {}
instances:
  s00:
    component: mmi1x2
    settings:
      cross_section: strip
      gap_mmi: 0.25
      length_mmi: 5.5
      length_taper: 10.0
      straight: straight
      taper: taper
      width_mmi: 2.5
      width_taper: 1.0
  s10:
    component: mmi1x2
    settings:
      cross_section: strip
      gap_mmi: 0.25
      length_mmi: 5.5
      length_taper: 10.0
      straight: straight
      taper: taper
      width_mmi: 2.5
      width_taper: 1.0
  s11:
    component: mmi1x2
    settings:
      cross_section: strip
      gap_mmi: 0.25
      length_mmi: 5.5
      length_taper: 10.0
      straight: straight
      taper: taper
      width_mmi: 2.5
      width_taper: 1.0
  s20:
    component: mmi1x2
    settings:
      cross_section: strip
      gap_mmi: 0.25
      length_mmi: 5.5
      length_taper: 10.0
      straight: straight
      taper: taper
      width_mmi: 2.5
      width_taper: 1.0
  s21:
    component: mmi1x2
    settings:
      cross_section: 

In [15]:
yaml.dump(s.netlist.model_dump(exclude_none=True), open("schematic.yaml", "w"))

## Python routing

In [20]:
n = 2**3
splitter = gf.components.splitter_tree(noutputs=n, spacing=(50, 50))
dbr_array = gf.components.array(
    component=gf.c.dbr, rows=n, columns=1, row_pitch=1, column_pitch=3, centered=True
)
s = gt.Schematic()
s.add_instance("s", gt.Instance(component=splitter))
s.add_instance("dbr", gt.Instance(component=dbr_array))
s.add_placement("s", gt.Placement(x=0, y=0))
s.add_placement("dbr", gt.Placement(x=300, y=0))

for i in range(n):
    s.add_net(
        gt.Net(
            p1=f"s,o2_2_{i+1}",
            p2=f"dbr,o1_{i+1}_1",
            name="splitter_to_dbr",
            settings=dict(radius=5, sort_ports=True),
        )
    )

g = s.netlist
g

Netlist(pdk='', instances={'s': Instance(component='splitter_tree', settings={'coupler': 'mmi1x2', 'noutputs': 8, 'spacing': (50, 50), 'bend_s': 'bend_s', 'cross_section': 'strip'}, info={}, array=None), 'dbr': Instance(component='array', settings={'component': 'dbr', 'columns': 1, 'rows': 8, 'column_pitch': 3, 'row_pitch': 1, 'add_ports': True, 'centered': True, 'auto_rename_ports': False}, info={}, array=None)}, placements={'s': Placement(x=0.0, y=0.0, xmin=None, ymin=None, xmax=None, ymax=None, dx=0, dy=0, port=None, rotation=0, mirror=False), 'dbr': Placement(x=300.0, y=0.0, xmin=None, ymin=None, xmax=None, ymax=None, dx=0, dy=0, port=None, rotation=0, mirror=False)}, connections={}, routes={'splitter_to_dbr': Bundle(links={'s,o2_2_1': 'dbr,o1_1_1', 's,o2_2_2': 'dbr,o1_2_1', 's,o2_2_3': 'dbr,o1_3_1', 's,o2_2_4': 'dbr,o1_4_1', 's,o2_2_5': 'dbr,o1_5_1', 's,o2_2_6': 'dbr,o1_6_1', 's,o2_2_7': 'dbr,o1_7_1', 's,o2_2_8': 'dbr,o1_8_1'}, settings={'radius': 5, 'sort_ports': True}, routing_s

In [21]:
dbr_array.pprint_ports()

┏━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name   ┃ width ┃ orientation ┃ layer    ┃ center       ┃ port_type ┃
┡━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ o1_1_1 │ 0.5   │ 180.0       │ WG (1/0) │ (-1.6, -3.5) │ optical   │
│ o1_2_1 │ 0.5   │ 180.0       │ WG (1/0) │ (-1.6, -2.5) │ optical   │
│ o1_3_1 │ 0.5   │ 180.0       │ WG (1/0) │ (-1.6, -1.5) │ optical   │
│ o1_4_1 │ 0.5   │ 180.0       │ WG (1/0) │ (-1.6, -0.5) │ optical   │
│ o1_5_1 │ 0.5   │ 180.0       │ WG (1/0) │ (-1.6, 0.5)  │ optical   │
│ o1_6_1 │ 0.5   │ 180.0       │ WG (1/0) │ (-1.6, 1.5)  │ optical   │
│ o1_7_1 │ 0.5   │ 180.0       │ WG (1/0) │ (-1.6, 2.5)  │ optical   │
│ o1_8_1 │ 0.5   │ 180.0       │ WG (1/0) │ (-1.6, 3.5)  │ optical   │
└────────┴───────┴─────────────┴──────────┴──────────────┴───────────┘

In [22]:
splitter.pprint_ports()

┏━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ name   ┃ width ┃ orientation ┃ layer    ┃ center         ┃ port_type ┃
┡━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ o1_0_0 │ 0.5   │ 180.0       │ WG (1/0) │ (-10.0, 0.0)   │ optical   │
│ o2_2_1 │ 0.5   │ 0.0         │ WG (1/0) │ (165.5, -62.5) │ optical   │
│ o2_2_2 │ 0.5   │ 0.0         │ WG (1/0) │ (165.5, -87.5) │ optical   │
│ o2_2_3 │ 0.5   │ 0.0         │ WG (1/0) │ (165.5, -12.5) │ optical   │
│ o2_2_4 │ 0.5   │ 0.0         │ WG (1/0) │ (165.5, -37.5) │ optical   │
│ o2_2_5 │ 0.5   │ 0.0         │ WG (1/0) │ (165.5, 37.5)  │ optical   │
│ o2_2_6 │ 0.5   │ 0.0         │ WG (1/0) │ (165.5, 12.5)  │ optical   │
│ o2_2_7 │ 0.5   │ 0.0         │ WG (1/0) │ (165.5, 87.5)  │ optical   │
│ o2_2_8 │ 0.5   │ 0.0         │ WG (1/0) │ (165.5, 62.5)  │ optical   │
└────────┴───────┴─────────────┴──────────┴────────────────┴───────────┘

In [23]:
yaml.dump(s.netlist.model_dump(exclude_none=True), open("schematic.yaml", "w"))
yaml_component = yaml.dump(s.netlist.model_dump(exclude_none=True))
print(yaml_component)

connections: {}
info: {}
instances:
  dbr:
    component: array
    settings:
      add_ports: true
      auto_rename_ports: false
      centered: true
      column_pitch: 3
      columns: 1
      component: dbr
      row_pitch: 1
      rows: 8
  s:
    component: splitter_tree
    settings:
      bend_s: bend_s
      coupler: mmi1x2
      cross_section: strip
      noutputs: 8
      spacing: !!python/tuple
      - 50
      - 50
nets: []
pdk: ''
placements:
  dbr:
    dx: 0
    dy: 0
    mirror: false
    rotation: 0
    x: 300.0
    y: 0.0
  s:
    dx: 0
    dy: 0
    mirror: false
    rotation: 0
    x: 0.0
    y: 0.0
ports: {}
routes:
  splitter_to_dbr:
    links:
      s,o2_2_1: dbr,o1_1_1
      s,o2_2_2: dbr,o1_2_1
      s,o2_2_3: dbr,o1_3_1
      s,o2_2_4: dbr,o1_4_1
      s,o2_2_5: dbr,o1_5_1
      s,o2_2_6: dbr,o1_6_1
      s,o2_2_7: dbr,o1_7_1
      s,o2_2_8: dbr,o1_8_1
    routing_strategy: route_bundle
    settings:
      radius: 5
      sort_ports: true
warnings: {}



In [24]:
import yaml

from gdsfactory.serialization import convert_tuples_to_lists

conf = s.netlist.model_dump(exclude_none=True)
conf = convert_tuples_to_lists(conf)
yaml_component = yaml.dump(conf)
print(yaml_component)

connections: {}
info: {}
instances:
  dbr:
    component: array
    settings:
      add_ports: true
      auto_rename_ports: false
      centered: true
      column_pitch: 3
      columns: 1
      component: dbr
      row_pitch: 1
      rows: 8
  s:
    component: splitter_tree
    settings:
      bend_s: bend_s
      coupler: mmi1x2
      cross_section: strip
      noutputs: 8
      spacing:
      - 50
      - 50
nets: []
pdk: ''
placements:
  dbr:
    dx: 0
    dy: 0
    mirror: false
    rotation: 0
    x: 300.0
    y: 0.0
  s:
    dx: 0
    dy: 0
    mirror: false
    rotation: 0
    x: 0.0
    y: 0.0
ports: {}
routes:
  splitter_to_dbr:
    links:
      s,o2_2_1: dbr,o1_1_1
      s,o2_2_2: dbr,o1_2_1
      s,o2_2_3: dbr,o1_3_1
      s,o2_2_4: dbr,o1_4_1
      s,o2_2_5: dbr,o1_5_1
      s,o2_2_6: dbr,o1_6_1
      s,o2_2_7: dbr,o1_7_1
      s,o2_2_8: dbr,o1_8_1
    routing_strategy: route_bundle
    settings:
      radius: 5
      sort_ports: true
warnings: {}



In [25]:
c = gf.read.from_yaml(yaml_component)
c

ValueError: Either cross_section=None or layer=None and route_width=None must be provided